# **ЗАДАНИЕ 9.5. МОДУЛЬ ML-6**

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. 

**КРИТЕРИИ ОЦЕНИВАНИЯ:**

* 1 балл - Верно выделены три столбца-признака для обучения, выбранные **RFE**.
* 1 балл - Верно выделены три столбца-признака для обучения, выбранные **SelectKBest**.
* 3 балла - Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
* 3 балла - Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
* 2 балла - Произведено сравнение выбранных метрик в форме комментария. Дан ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» (в текстовой ячейке).

### **Предобработка данных**

In [13]:
# блок с библиотеками
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE, SelectKBest, f_regression

In [7]:
# загрузка датасета из папки data (метод !wget не отрабатывает)
data = pd.read_excel('data/data_ford_price.xlsx') 

In [74]:
# выделение тестовой и тренировочной выборок
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

### **Сырое предсказание**

In [75]:
estimator = LinearRegression()
estimator.fit(X_train, y_train)
y_predicted = estimator.predict(X_test)
print('MAE score with SelectKBest: {:.3f} $.'.format(metrics.mean_absolute_error(y_test, y_predicted)))

MAE score with SelectKBest: 4682.957 $.


### **RFE**

In [76]:
# выполняем отбор признаков методом RFE
estimator = LinearRegression()
selector_RFE = RFE(estimator, n_features_to_select=3, step=1)
selector_RFE = selector_RFE.fit(X_train, y_train)

In [77]:
# Делаем предсказание и выводим метрику МАЕ
y_pred_test = selector_RFE.predict(X_test)
print('MAE score with RFE: {:.3f} $.'.format(metrics.mean_absolute_error(y_test, y_pred_test)))

MAE score with RFE: 5096.570 $.


### **SelectKBest**

In [60]:
# выполняем отбор признаков методом SelectKBest
selector_KBest = SelectKBest(f_regression, k=3)
selector_KBest.fit_transform(X_train, y_train)
needed_features = selector_KBest.get_feature_names_out()

In [61]:
# фильтруем ненужные признаки для обучения без них
X_train_new = X_train[needed_features]
X_test_new = X_test[needed_features]

In [69]:
# Делаем обучение, предсказание и выводим метрику МАЕ
estimator.fit(X_train_new, y_train)
y_predicted = estimator.predict(X_test_new)
print('MAE score with SelectKBest: {:.3f} $.'.format(metrics.mean_absolute_error(y_test, y_predicted)))

MAE score with SelectKBest: 4708.946 $.


## **ВЫВОД**

Модель, обученная на признаках, отобранных методом SelectKBest показала лучший результат чем модель, обученная на признаках, отобранных методом RFE. Средняя абсолютная ошибка меньше в SelectKBest на 388 $. 

Однако следует уточнить, что относительно "сырого" предсказания (на изначальном датасете), метрики ухудшились в обоих случаях.